In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pybedtools import BedTool
from Bio import SeqIO
from Bio.Seq import Seq
import os
import sys
from shutil import copyfile
from tqdm import tqdm, tqdm_notebook
import multiprocessing
from multiprocessing import Pool

In [2]:
events = os.listdir('../data/norm_splice/')

In [3]:
encore_dir = '../data/encore_events'
for d in os.listdir(encore_dir):
    if d.endswith('HepG2'):
        rbp_name = d.split('-')[0]
        
        for event in events:
            orig_file = '{}/{}/MATS_Norm_output/{}.MATS.JunctionCountOnly.txt'.format(encore_dir, d, event)
            copy_file = '../data/norm_splice/{}/{}.MATS.JunctionCountOnly.txt'.format(event, rbp_name)
            copyfile(orig_file, copy_file)

FileNotFoundError: [Errno 2] No such file or directory: '../data/encore_events'

# Feature engineering

`?*4 nt` sequence of exon-exon junction windows.

`? nt` windows flanking the relevant exon/intron boundaries, extending a maximum of `? nt` into each exon and `? nt` into each intron.

Feature windows:
- upstream exon window
- cassette exon 5' window
- cassette exon 3' window
- downstream exon window

In [105]:
def generate_features(raw_filepath):
    data = pd.read_csv('../data/norm_splice/SE/{}'.format(raw_filepath), sep='\t')
    bg_data = data.loc[data['FDR'] > 0.3]
    bg_data = bg_data.iloc[np.random.randint(bg_data.shape[0], size=1)]
    
    data = data.loc[(data['FDR'] < 0.1) & (data['IncLevelDifference'] < 0)]
    data = pd.concat([bg_data, data])
    
    rbp_name = raw_filepath.split('.')[0]
    exon_overlap = 50
    intron_overlap = 350

    upstream_file = open(f'../data/features/SE/{rbp_name}.upstream.bed', 'w')
    cassette_5p_file = open(f'../data/features/SE/{rbp_name}.cassette_5p.bed', 'w')
    cassette_3p_file = open(f'../data/features/SE/{rbp_name}.cassette_3p.bed', 'w')
    downstream_file = open(f'../data/features/SE/{rbp_name}.downstream.bed', 'w')
    
    for i, row in data.iterrows():
        # upstream exon 3' region
        upstream_3p_start = max(row['upstreamES'], row['upstreamEE'] - exon_overlap)
        upstream_3p_end = min(row['exonStart_0base'], row['upstreamEE'] + intron_overlap)
        upstream = [upstream_3p_start, upstream_3p_end]

        # cassette exon 5' region
        cassette_5p_start = max(row['upstreamEE'], row['exonStart_0base'] - intron_overlap)
        cassette_5p_end = min(row['exonStart_0base'] + exon_overlap, row['exonEnd'])
        cassette_5p = [cassette_5p_start, cassette_5p_end]

        # cassette exon 3' region
        cassette_3p_start = max(row['exonEnd'] - exon_overlap, row['exonStart_0base'])
        cassette_3p_end = min(row['downstreamES'], row['exonEnd'] + intron_overlap)
        cassette_3p = [cassette_3p_start, cassette_3p_end]

        # downstream exon 5' window
        downstream_5p_start = max(row['exonEnd'], row['downstreamES'] - intron_overlap)
        downstream_5p_end = min(row['downstreamES'] + exon_overlap, row['downstreamEE'])
        downstream = [downstream_5p_start, downstream_5p_end]

        # Make sure windows are defined 5' -> 3'
        assert (upstream[0] < upstream[1]) 
        assert (cassette_5p[0] < cassette_5p[1]) 
        assert (cassette_3p[0] < cassette_3p[1]) 
        assert (downstream[0] < downstream[1]) 

        chr = row['chr']

        # write bed sequences
        upstream_file.write(f'{chr}\t{upstream[0]}\t{upstream[1]}\t{row["FDR"]}\n')
        cassette_5p_file.write(f'{chr}\t{cassette_5p[0]}\t{cassette_5p[1]}\t{row["FDR"]}\n')
        cassette_3p_file.write(f'{chr}\t{cassette_3p[0]}\t{cassette_3p[1]}\t{row["FDR"]}\n')
        downstream_file.write(f'{chr}\t{downstream[0]}\t{downstream[1]}\t{row["FDR"]}\n')

In [107]:
p = Pool(8)

se_files = os.listdir('../data/norm_splice/SE')
total = len(se_files)

for _ in tqdm_notebook(p.imap_unordered(generate_features, se_files), total=total):
        pass

Aggregate into 1 file

In [108]:
feature_dir = '../data/features/SE'
output_suffix = ['upstream.bed', 'cassette_5p.bed', 'cassette_3p.bed', 'downstream.bed']

for out_suffix in output_suffix:
    out_feature_file = f'{feature_dir}_{out_suffix}'
    if os.path.exists(out_feature_file):
        os.remove(out_feature_file)
    
for file in tqdm_notebook(sorted(os.listdir(feature_dir))):
    if file == '.ipynb_checkpoints':
        continue
        
    filepath = os.path.join(feature_dir, file)
    if os.path.getsize(filepath) > 1:
        data = pd.read_csv(filepath, sep='\t', header=None)

        for out_suffix in output_suffix:
            if filepath.endswith(out_suffix):
                data[4] = file.split('.')[0]
                
#                 if file.split('.')[0] in keep:
                
                data.loc[data.iloc[:, 3] > 0.1, 4] = 'negative'
                data = data.drop(axis=1, columns=3)
                data.to_csv(f'{feature_dir}_{out_suffix}', sep='\t', header=False, index=False, mode='a')        
                break

## Event filtering

### Read in all features as single matrix

In [153]:
a = []
for out_suffix in output_suffix:
    if out_suffix == 'downstream.bed':
        a.append(pd.read_csv('_'.join([feature_dir, out_suffix]), sep='\t', header=None))
    else:
        a.append(pd.read_csv('_'.join([feature_dir, out_suffix]), sep='\t', header=None).iloc[:, :-1])

In [154]:
events = pd.concat(a, axis=1)
events.columns = list(range(events.shape[1]))
events.shape

(7187, 13)

### Drop events

In [155]:
events = events.drop_duplicates(subset=events.columns[:-1], keep=False)

### Subset events to most common classes

In [156]:
events[12].value_counts()

U2AF2       221
negative    209
U2AF1       173
PABPC1       88
HNRNPC       79
SNRNP70      77
MAGOH        72
SRFBP1       64
PUF60        58
SRSF1        54
SF3A3        53
SRSF3        44
RRP9         43
EWSR1        42
DDX19B       34
RPL23A       34
SF3B1        26
KHSRP        25
RAVER1       23
HNRNPU       21
UCHL5        20
EIF4A3       19
UBE2L3       17
SRSF9        16
SF3B4        15
PRPF6        14
EIF4G1       14
HNRNPL       13
RBM34        13
FTO          13
           ... 
TBRG4         1
G3BP2         1
RBM47         1
ACO1          1
PSIP1         1
DNAJC2        1
PCBP1         1
CUGBP1        1
NIP7          1
AUH           1
HNRNPA1       1
TFIP11        1
PPIG          1
G3BP1         1
RPS19         1
RCC2          1
CCAR1         1
GRWD1         1
ABCF1         1
HNRNPUL1      1
RBM27         1
TARDBP        1
XRN2          1
PA2G4         1
PKM2          1
MATR3         1
SUGP2         1
RBFOX2        1
PUS1          1
PRPF4         1
Name: 12, Length: 172, d

In [149]:
keep = events[12].value_counts()[:25].index
# keep = keep.drop('negative') # remove negative controls
events = events.loc[events[12].isin(keep)]
events[12].value_counts()

U2AF2       221
negative    209
U2AF1       173
PABPC1       88
HNRNPC       79
SNRNP70      77
MAGOH        72
SRFBP1       64
PUF60        58
SRSF1        54
SF3A3        53
SRSF3        44
RRP9         43
EWSR1        42
DDX19B       34
RPL23A       34
SF3B1        26
KHSRP        25
RAVER1       23
HNRNPU       21
UCHL5        20
EIF4A3       19
UBE2L3       17
SRSF9        16
SF3B4        15
Name: 12, dtype: int64

In [152]:
events.shape[0] - (221 + 209+173)

924

### Write de-duped events to file

In [150]:
feature_cols = [[0,1,2], [3,4,5], [6,7,8], [9,10,11]]
feature_cols = [cols + [-1] for cols in feature_cols]

for out_suffix, cols in zip(output_suffix, feature_cols):
    out_feature_file = f'{feature_dir}_25class_dedup_{out_suffix}'

    if os.path.exists(out_feature_file):
        os.remove(out_feature_file)

    events.iloc[:, cols].to_csv(out_feature_file, sep='\t', header=False, index=False, mode='a')